In [1]:
import pandas as pd
from scipy.stats import fisher_exact
from analytics_utils.db_utils import get_db_client
import warnings
warnings.filterwarnings('ignore')
from statsmodels.stats.proportion import proportions_ztest
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from collections import defaultdict
from scipy.stats import ttest_ind

In [2]:
db = get_db_client(environment='production', account_id=None, db_type='bigquery')

Found credentials in shared credentials file: ~/.aws/credentials


In [3]:
sql = """
SELECT
    (CASE WHEN dim_application.meets_required_certifications  THEN 'Yes' ELSE 'No' END) AS dim_application_meets_required_certifications,
    (CASE WHEN dim_application.meets_required_education  THEN 'Yes' ELSE 'No' END) AS dim_application_meets_required_education,
    (CASE WHEN dim_application.meets_required_languages  THEN 'Yes' ELSE 'No' END) AS dim_application_meets_required_languages,
    (CASE WHEN dim_application.meets_required_min_industry_experience  THEN 'Yes' ELSE 'No' END) AS dim_application_meets_required_min_industry_experience,
    (CASE WHEN dim_application.meets_required_min_relevant_experience  THEN 'Yes' ELSE 'No' END) AS dim_application_meets_required_min_relevant_experience,
    (CASE WHEN dim_application.meets_required_relocation  THEN 'Yes' ELSE 'No' END) AS dim_application_meets_required_relocation,
    (CASE WHEN dim_application.meets_required_security_clearance  THEN 'Yes' ELSE 'No' END) AS dim_application_meets_required_security_clearance,
    (CASE WHEN dim_application.meets_required_skills  THEN 'Yes' ELSE 'No' END) AS dim_application_meets_required_skills,
    (CASE WHEN dim_application.meets_required_visa  THEN 'Yes' ELSE 'No' END) AS dim_application_meets_required_visa,
    dim_application.customer AS dim_application_customer, 
    dim_req.country_code  AS dim_req_country_code,
    dim_req.required_seniority_level  AS dim_req_required_seniority_level,
    dim_application.country_code  AS dim_application_country_code,
    dim_application.resume_language  AS dim_application_resume_language,
    (CASE WHEN dim_application.has_resume  THEN 'Yes' ELSE 'No' END) AS dim_application_has_resume,
    dim_application.resume_language  AS dim_application_resume_language,
    (CASE WHEN dim_req.is_required_certifications  THEN 'Yes' ELSE 'No' END) AS dim_req_is_required_certifications,
    (CASE WHEN dim_application.is_internal  THEN 'Yes' ELSE 'No' END) AS dim_application_is_internal,
    CASE
WHEN (case when dim_application.original_spotlight_grade in ('A','B','C','D') then dim_application.original_spotlight_grade else 'N/A' end) in ('A','B')  THEN '0'
WHEN (case when dim_application.original_spotlight_grade in ('A','B','C','D') then dim_application.original_spotlight_grade else 'N/A' end) in ('C','D')  THEN '1'
ELSE '2'
END AS dim_application_priority_grade__sort_,
    CASE
WHEN (case when dim_application.original_spotlight_grade in ('A','B','C','D') then dim_application.original_spotlight_grade else 'N/A' end) in ('A','B')  THEN 'Priority'
WHEN (case when dim_application.original_spotlight_grade in ('A','B','C','D') then dim_application.original_spotlight_grade else 'N/A' end) in ('C','D')  THEN 'Non-Priority'
ELSE 'No Grade'
END AS dim_application_priority_grade,
    dim_application.gender  AS dim_application_gender,
    case when dim_application.race in ('Asian','Black or African American','Hispanic or Latino','White','Two or more races') then dim_application.race
              when dim_application.race in ('Not Applicable','Not applicable','undefined') or dim_application.race is null then null
              when dim_application.race in ('Decline','Decline to Identify','Not Declared','Not declared','Prefer Not to Say','Do not wish to disclose','Dont Wish To Answer') then 'Decline to Identify'
              else "Other" end AS dim_application_race_bucketed,
    ROUND(dim_application.total_years_of_experience) AS dim_application_total_years_of_experience,
    dim_application.highest_degree  AS dim_application_highest_degree,
        (CASE WHEN dim_application.meets_all_requirements  THEN 'Yes' ELSE 'No' END) AS dim_application_meets_all_requirements,
    dim_req.required_education  AS dim_req_required_education,
    case when dim_req.top_category = 'all' or dim_req.top_category is null then "to exclude" else dim_req.top_category end AS dim_req_top_category_filter_only_relevant_values,
    fact_ai_raw_grading.min_years_of_relevant_experience AS fact_ai_raw_grading_min_years_of_relevant_experience,
    ROUND(fact_ai_raw_grading.final_relevant_experience_years) AS fact_ai_raw_grading_final_relevant_experience_years,
    fact_ai_raw_grading.conditional_req_min_years_of_relevant_experience  AS fact_ai_raw_grading_conditional_req_min_years_of_relevant_experience,
    COUNT(DISTINCT  dim_application.key ) AS count_applications
-- FROM `hs-analytics.all_customers.dwh_hs_fact_application_status_history` AS fact_application_status_history
-- LEFT JOIN `hs-analytics.all_customers.dwh_hs_dim_application`
--     AS dim_application ON fact_application_status_history.business_application_key = dim_application.key
--      and dim_application.customer = fact_application_status_history.customer
FROM `hs-analytics.all_customers.dwh_hs_dim_application` AS dim_application
LEFT JOIN `hs-analytics.all_customers.dwh_hs_dim_req`
     AS dim_req ON dim_application.req_key = dim_req.key
      and dim_req.customer = dim_application.customer
LEFT JOIN `hs-analytics.all_customers.dwh_hs_fact_ai_grading_results`
     AS fact_ai_raw_grading ON fact_ai_raw_grading.application_key = dim_application.key
      and fact_ai_raw_grading.customer = dim_application.customer
WHERE 
      dim_application.date_applied  >= TIMESTAMP('2024-08-01 00:00:00') AND 
      dim_application.customer in (select distinct customer from `hs-analytics.all_customers.dwh_hs_dim_source_systems` where is_live_client is true or customer in ('aden','cancun','athens','miami','derby','potsdam')) AND 
      case when dim_application.original_spotlight_grade in ('A','B','C','D') then dim_application.original_spotlight_grade else 'N/A' end  IS NOT NULL AND
      dim_application.country_code = 'USA' AND 
      dim_req.country_code = 'USA' AND 
      dim_req.is_gradable 
AND RAND() < 0.05
GROUP BY
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,

    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30
ORDER BY
    1
"""

In [4]:
res = db.execute(sql)

In [ ]:
res_df = pd.DataFrame([dict(row) for row in res['query_reults']])
len(res_df)

In [ ]:
res_df = res_df[res_df['dim_application_priority_grade'] != 'No Grade']
len(df), res_df['count_applications'].sum()

In [ ]:
res_df['is_priority'] = res_df['dim_application_priority_grade'] == 'Priority'

In [ ]:
df_expanded = res_df.loc[df.index.repeat(res_df['count_applications'])]
len(df_expanded)

In [ ]:
EEO_FIELDS = ['dim_application_gender', 'dim_application_race_bucketed']
MIN_PERCENT_FOR_GROUP = 0.02

TO_ADJUST = ['dim_application_total_years_of_experience', 
      'dim_application_highest_degree', 
      'dim_req_required_education', 
      'dim_req_top_category_filter_only_relevant_values',
      'fact_ai_raw_grading_min_years_of_relevant_experience',
      'dim_req_required_seniority_level',
      'dim_application_country_code',
      'dim_application_resume_language',
      'dim_application_has_resume',
      'dim_req_is_required_certifications',
      'dim_application_is_internal',
      'fact_ai_raw_grading_final_relevant_experience_years', 
      'fact_ai_raw_grading_conditional_req_min_years_of_relevant_experience',
     ]

TO_ADJUST_QUALS = [
   'dim_application_meets_required_certifications',
   'dim_application_meets_required_education',
   'dim_application_meets_required_languages',
   'dim_application_meets_required_min_industry_experience',
   'dim_application_meets_required_min_relevant_experience',
   'dim_application_meets_required_relocation',
   'dim_application_meets_required_security_clearance',
   'dim_application_meets_required_skills',
   'dim_application_meets_required_visa',
   'dim_application_is_internal',
    'dim_application_resume_language',
    'dim_req_top_category_filter_only_relevant_values',
]


def sample_equal(df_bin, eeo_field):
    if df_bin[eeo_field].nunique() < 2:
        min_count = 0
    else:
        min_count = df_bin.groupby(eeo_field).size().min()
    sampled_groups = df_bin.groupby(eeo_field).apply(lambda x: x.sample(n=min_count, replace=False, random_state=42))
    return sampled_groups.droplevel(0)

In [ ]:
dfs = {}
images = defaultdict(list)
for eeo_field in EEO_FIELDS:
    value_counts = df_expanded[eeo_field].value_counts()
    relevant_groups = set(value_counts[value_counts > len(df_expanded)*MIN_PERCENT_FOR_GROUP].index) - {'Decline to Identify'}
    relevant_groups_df = df_expanded[df_expanded[eeo_field].isin(relevant_groups)]
    relevant_groups_means = relevant_groups_df.groupby(eeo_field)['is_priority'].mean()
    max_group  = relevant_groups_df.groupby(eeo_field)['is_priority'].mean().idxmax()
    
    for group in relevant_groups - {max_group}:
        print(eeo_field, group)

        group_df = relevant_groups_df[relevant_groups_df[eeo_field].isin({group, max_group})]
        group_df_after = group_df.groupby(TO_ADJUST).apply(lambda x: sample_equal(x, eeo_field)).reset_index(drop=True)
        
        value_counts_after = group_df_after[eeo_field].value_counts()
        groups_means_after = group_df_after.groupby(eeo_field)['is_priority'].mean()
          
        group_counts = group_df.groupby(eeo_field)['is_priority'].agg(['sum', 'count'])
        count = group_counts['sum'].values  
        nobs = group_counts['count'].values 
        ztest_before = proportions_ztest(count, nobs)

        group_a = group_df[group_df[eeo_field] == max_group]['is_priority']
        group_b = group_df[group_df[eeo_field] == group]['is_priority']
        t_stat, t_pval = ttest_ind(group_a, group_b)
        
        group_table_after = group_df_after.groupby([eeo_field, 'is_priority'])['count_applications'].sum().unstack()
        group_counts_after = group_df_after.groupby(eeo_field)['is_priority'].agg(['sum', 'count'])
        count_after = group_counts_after['sum'].values  
        nobs_after = group_counts_after['count'].values 
        ztest_after = proportions_ztest(count_after, nobs_after)

        group_a_after = group_df_after[group_df_after[eeo_field] == max_group]['is_priority']
        group_b_after = group_df_after[group_df_after[eeo_field] == group]['is_priority']
        t_stat_after, t_pval_after = ttest_ind(group_a_after, group_b_after)

        
        group_report_data = [{
            'group': max_group,
            'num_applicants_before': value_counts[max_group],
            'positave_rate_before': round(groups_means_after[max_group], 4),
            'z_test_p_value_before': 'NA',
            't_test_p_value_before': 'NA', 
            'num_applicants_after': value_counts_after[max_group],
            'percent_applicants_after': round(value_counts_after[max_group] / value_counts[max_group] * 100, 2),
            'positave_rate_after': round(groups_means_after[max_group], 4),
            'z_test_p_value_after': 'NA',
            't_test_p_value_after': 'NA',
        }]


        group_report_data.append({
            'group': group,
            'num_applicants_before': value_counts[group],
            'positave_rate_before': round(groups_means_after[group], 4),
            'z_test_p_value_before': round(ztest_before[1], 6),
            't_test_p_value_before': round(t_pval, 6),
            'num_applicants_after': value_counts_after[group],
            'percent_applicants_after': round(value_counts_after[group] / value_counts[group] * 100, 2),
            'positave_rate_after': round(groups_means_after[group], 4),
            'z_test_p_value_after': round(ztest_after[1], 6),
            't_test_p_value_after': round(t_pval_after, 6),
        })
        
        dfs[f'{group} vs {max_group}'] = pd.DataFrame(group_report_data)


len(dfs)

In [ ]:
sorted(dfs.keys())

In [ ]:
for df in dfs.values():
    display(df)

In [ ]:
meta_data = [{
    'clients': df['dim_application_customer'].unique(),
    'countries': df['dim_application_country_code'].unique(),
    'from_date': datetime(2024, 8, 1),
    'to_date': datetime(2024, 11, 1),
    'total_number_of_applications': df['count_applications'].sum()
}]

meta_data_df = pd.DataFrame(meta_data).T

In [ ]:
with pd.ExcelWriter('general_bias_test_report_31_10_2024.xlsx', engine='openpyxl') as writer:
    meta_data_df.to_excel(writer, sheet_name='Meta Data', header=False)
    test_results_df.to_excel(writer, sheet_name='Bias Test', index=False)

    for eeo_field in EEO_FIELDS:
        wb = writer.book
        ws = wb.create_sheet(title=f'{eeo_field} dimensions')
        for i, img_path in enumerate(images[eeo_field]):
            pos = f"A{1 + i * 30}"  
            img = Image(img_path)
            _ = ws.add_image(img, pos)

In [ ]:
test_results_df